# SuperDARN GPU Processing Visualization Tutorial

This notebook demonstrates the comprehensive visualization system for SuperDARN data processing with GPU acceleration.

## Key Features:
- **Real-time processing monitoring** - See what's happening during GPU processing
- **Interactive data exploration** - Navigate through time, beams, and parameters  
- **Scientific visualization** - Publication-ready plots of SuperDARN data
- **Performance dashboards** - GPU utilization and throughput monitoring
- **Quality assessment** - Automated data quality metrics

Perfect for scientists who want to understand and optimize their SuperDARN processing workflows.

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import SuperDARN GPU package
sys.path.insert(0, '..')
import superdarn_gpu as sd

# Import visualization components
from superdarn_gpu.visualization import (
    plot_range_time, plot_fan, plot_acf, 
    create_processing_dashboard, create_validation_dashboard,
    InteractiveExplorer, ParameterTuner, ProcessingComparison,
    RealtimeMonitor, PerformanceDashboard
)

# Setup for notebook display
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

print(f"🚀 SuperDARN GPU v{sd.__version__}")
print(f"📊 Visualization system loaded")
print(f"🔧 Backend: {sd.get_backend().__name__}")

if sd.GPU_AVAILABLE:
    print("✅ GPU acceleration available")
else:
    print("⚠️  Running on CPU - install CuPy for GPU acceleration")

## 1. Generate Example SuperDARN Data

First, let's create some realistic synthetic SuperDARN data that mimics real observations:

In [ ]:
def create_example_superdarn_data():
    """Create realistic SuperDARN data for demonstration"""
    print("🔬 Creating example SuperDARN data...")
    
    # Data dimensions
    n_times = 60    # 2 hours at 2-minute resolution
    n_beams = 16    # Standard SuperDARN beam pattern
    n_ranges = 75   # Range gates out to ~1300 km
    
    # Get array backend (CuPy for GPU, NumPy for CPU)
    xp = sd.get_backend()
    
    # Initialize arrays
    velocity = xp.full((n_times, n_beams, n_ranges), xp.nan, dtype=xp.float32)
    power = xp.full((n_times, n_beams, n_ranges), xp.nan, dtype=xp.float32)
    width = xp.full((n_times, n_beams, n_ranges), xp.nan, dtype=xp.float32)
    elevation = xp.full((n_times, n_beams, n_ranges), xp.nan, dtype=xp.float32)
    
    # Create realistic patterns
    ranges = xp.arange(180, 180 + n_ranges * 15, 15)  # km
    times = xp.arange(n_times) * 2  # minutes
    beams = xp.arange(n_beams)
    
    # Simulate ionospheric convection pattern
    for t in range(n_times):
        for b in range(n_beams):
            for r in range(n_ranges):
                range_km = ranges[r]
                
                # Distance-dependent backscatter probability
                scatter_prob = 0.8 * xp.exp(-(range_km - 600)**2 / (2 * 300**2))
                
                if xp.random.random() < scatter_prob:
                    # Convection velocity pattern
                    base_velocity = 400 * xp.sin(2 * xp.pi * t / 30) * xp.cos(xp.pi * b / n_beams)
                    velocity[t, b, r] = base_velocity + xp.random.normal(0, 80)
                    
                    # Power with range dependence
                    base_power = 35 - 0.02 * (range_km - 400)
                    power[t, b, r] = base_power + xp.random.normal(0, 5)
                    
                    # Spectral width
                    width[t, b, r] = xp.random.uniform(80, 300)
                    
                    # Elevation angle
                    elevation[t, b, r] = xp.random.uniform(10, 40)
    
    # Package data
    data = {
        'velocity': velocity,
        'power': power,
        'width': width,
        'elevation': elevation,
        'time': [datetime.now() + timedelta(minutes=2*i) for i in range(n_times)],
        'beam': beams,
        'range': ranges,
        'metadata': {
            'radar': 'sas',  # Saskatoon
            'date': datetime.now().date(),
            'synthetic': True
        }
    }
    
    # Calculate data completeness
    completeness = (~xp.isnan(velocity)).sum() / velocity.size * 100
    print(f"  📊 Data shape: {velocity.shape}")
    print(f"  ✅ Data completeness: {completeness:.1f}%")
    
    return data

# Generate the example data
superdarn_data = create_example_superdarn_data()

## 2. Basic Scientific Visualization

Let's start with basic scientific plots to visualize our SuperDARN data:

In [ ]:
# Create a comprehensive summary plot
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('SuperDARN Data Overview', fontsize=16)

# Velocity range-time plot
plot_range_time(superdarn_data, parameter='velocity', ax=axes[0,0], 
               title='Line-of-sight Velocity')

# Power range-time plot  
plot_range_time(superdarn_data, parameter='power', ax=axes[0,1],
               title='Backscatter Power')

# Spectral width
plot_range_time(superdarn_data, parameter='width', ax=axes[1,0],
               title='Spectral Width')

# Fan plot at a specific time
plot_fan(superdarn_data, parameter='velocity', time_idx=30, ax=axes[1,1],
        title='Velocity Fan Plot (t=30)')

plt.tight_layout()
plt.show()

print("📈 Basic scientific visualizations complete!")

## 3. Interactive Data Explorer

Now let's create an interactive explorer that lets scientists navigate through the data:

In [ ]:
# Create interactive explorer
print("🔧 Creating interactive data explorer...")
print("   Use the sliders to explore different times, beams, and parameters")
print("   Click 'Play' to animate through time")

explorer = InteractiveExplorer(superdarn_data, 
                              parameters=['velocity', 'power', 'width'])

plt.show()

print("✨ Interactive explorer launched!")
print("   • Time slider: Navigate through observation period")
print("   • Beam slider: Select specific radar beam")
print("   • Parameter radio buttons: Switch between velocity/power/width")
print("   • Play button: Auto-animate through time")

## 4. Processing Pipeline Visualization

Let's simulate processing the data and visualize each stage:

In [ ]:
from superdarn_gpu.visualization import ProcessingViewer
import time

def simulate_processing_stages(data):
    """Simulate different processing stages for visualization"""
    
    print("🔄 Simulating SuperDARN processing pipeline...")
    
    # Create processing viewer
    viewer = ProcessingViewer()
    
    # Stage 1: Raw data
    print("  📡 Stage 1: Raw data loading")
    viewer.add_processing_stage("Raw Data Input", data)
    time.sleep(1)
    
    # Stage 2: ACF calculation (simulated)
    print("  🔍 Stage 2: ACF calculation")
    xp = sd.get_backend()
    n_times, n_beams, n_ranges = data['velocity'].shape
    
    # Simulate complex ACF data
    acf_data = xp.zeros((n_times, n_beams, n_ranges, 20), dtype=xp.complex64)
    for lag in range(20):
        decay = xp.exp(-lag * 0.15)
        phase = 2 * xp.pi * data['velocity'] * lag * 2400e-6 / 15e3
        magnitude = data['power'] * decay / 20  # Scale down power
        acf_data[:, :, :, lag] = magnitude * xp.exp(1j * phase)
    
    acf_results = {'acf': acf_data, 'power': data['power']}
    viewer.add_processing_stage("ACF Calculation", acf_results)
    time.sleep(1)
    
    # Stage 3: FitACF processing
    print("  🎯 Stage 3: FitACF processing")
    # Add some processing noise/refinement
    processed_velocity = data['velocity'] + xp.random.normal(0, 10, data['velocity'].shape)
    processed_data = {
        'velocity': processed_velocity,
        'power': data['power'] + xp.random.normal(0, 1, data['power'].shape),
        'width': data['width'],
        'velocity_error': xp.random.uniform(20, 100, data['velocity'].shape)
    }
    viewer.add_processing_stage("FitACF Results", processed_data)
    time.sleep(1)
    
    # Stage 4: Quality control
    print("  ✅ Stage 4: Quality control")
    quality_data = processed_data.copy()
    # Remove obviously bad data
    bad_mask = (xp.abs(processed_velocity) > 2000) | (data['power'] < 10)
    quality_data['velocity'] = xp.where(bad_mask, xp.nan, processed_velocity)
    viewer.add_processing_stage("Quality Filtered", quality_data)
    
    print("✅ Processing simulation complete!")
    return viewer

# Run processing simulation
processing_viewer = simulate_processing_stages(superdarn_data)

plt.show()

print("🎬 Processing pipeline viewer launched!")
print("   • Use navigation buttons to step through processing stages")
print("   • Watch how data changes at each step")
print("   • Understand the effect of each processing stage")

## 5. Performance Monitoring

Monitor GPU performance and processing efficiency:

In [ ]:
from superdarn_gpu.visualization import PerformanceDashboard, GPUMonitor

# GPU status check
gpu_monitor = GPUMonitor()
gpu_monitor.print_gpu_status()

# Create performance dashboard
print("📊 Creating performance monitoring dashboard...")
dashboard = PerformanceDashboard(update_interval=1.0)

# Simulate some processing with performance logging
print("🔄 Simulating processing workload for performance analysis...")

import time
dashboard.start_monitoring()

# Simulate different processing stages
stages = [
    ("Data Loading", 150, 45.2),
    ("ACF Calculation", 320, 12.8),  
    ("FitACF Processing", 180, 8.4),
    ("Quality Control", 80, 5.1),
    ("Data Export", 200, 15.3)
]

for stage_name, duration_ms, data_size_mb in stages:
    print(f"  Processing {stage_name}...")
    time.sleep(0.5)  # Simulate processing time
    dashboard.log_stage_performance(stage_name, duration_ms, data_size_mb)
    
print("⏱️  Processing simulation complete")

# Show performance dashboard
plt.show()

# Generate performance report
dashboard.generate_performance_report()
dashboard.stop_monitoring()

print("📈 Performance analysis complete!")

## 6. Comprehensive Processing Dashboard

Create a complete dashboard showing all processing results:

In [ ]:
# Prepare processing results
processing_results = {
    'raw_data': superdarn_data,
    'acf': {
        'magnitude': sd.get_backend().random.random((60, 16, 75, 20)),
        'phase': sd.get_backend().random.uniform(-3.14, 3.14, (60, 16, 75, 20))
    },
    'fitacf': {
        'velocity': superdarn_data['velocity'],
        'power': superdarn_data['power'],
        'width': superdarn_data['width']
    },
    'quality': {
        'data_completeness': 0.78,
        'velocity_quality': 0.85,
        'power_quality': 0.92,
        'temporal_consistency': 0.81
    }
}

print("🎨 Creating comprehensive processing dashboard...")

# Create the dashboard
dashboard_fig = create_processing_dashboard(
    superdarn_data, 
    processing_results,
    title="SuperDARN Processing Dashboard - Complete Analysis"
)

plt.show()

print("🎉 Comprehensive dashboard created!")
print("   This dashboard shows:")
print("   • Raw data visualization (velocity, power, spectral width)")
print("   • ACF analysis (magnitude, phase, spectrum)")
print("   • Processing quality metrics")
print("   • Statistical analysis and correlations")
print("   • Processing timeline and performance")

## 7. Parameter Tuning and Comparison

Interactive parameter tuning to optimize processing:

In [ ]:
# Simulate different processing algorithms for comparison
def algorithm_v1(data, threshold=0.1, smoothing=0.5):
    """Simulated processing algorithm v1"""
    result = data.copy()
    xp = sd.get_backend()
    
    # Apply threshold filtering
    mask = xp.abs(data['velocity']) < (threshold * 1000)
    result['velocity'] = xp.where(mask, xp.nan, data['velocity'])
    
    # Apply smoothing
    if smoothing > 0:
        # Simple temporal smoothing
        for t in range(1, data['velocity'].shape[0]-1):
            result['velocity'][t] = (1-smoothing) * data['velocity'][t] + \
                                   smoothing/2 * (data['velocity'][t-1] + data['velocity'][t+1])
    
    return result

def algorithm_v2(data, threshold=0.2, noise_filter=True):
    """Simulated processing algorithm v2"""
    result = data.copy()
    xp = sd.get_backend()
    
    # Different threshold approach
    power_threshold = 15 + threshold * 20
    mask = data['power'] < power_threshold
    result['velocity'] = xp.where(mask, xp.nan, data['velocity'])
    
    # Noise filtering
    if noise_filter:
        noise_mask = xp.abs(data['velocity']) > 1500  # Remove extreme velocities
        result['velocity'] = xp.where(noise_mask, xp.nan, result['velocity'])
    
    return result

# Create comparison of different algorithms
print("🔧 Creating algorithm comparison...")

algorithms = [
    lambda data: algorithm_v1(data, threshold=0.1, smoothing=0.3),
    lambda data: algorithm_v2(data, threshold=0.15, noise_filter=True),
    lambda data: superdarn_data  # Original data as baseline
]

labels = ['Algorithm v1 (Smoothed)', 'Algorithm v2 (Filtered)', 'Original Data']

comparison = ProcessingComparison(superdarn_data, algorithms, labels)

plt.show()

print("⚖️  Algorithm comparison complete!")
print("   Compare different processing approaches side-by-side")
print("   Evaluate which algorithm works best for your data")

## 8. Data Quality Validation

Comprehensive validation of processing results:

In [ ]:
from superdarn_gpu.visualization import ValidationViewer

# Create processed version of data for validation
processed_data = superdarn_data.copy()
xp = sd.get_backend()

# Add some processing effects
processed_data['velocity'] = superdarn_data['velocity'] + xp.random.normal(0, 5, superdarn_data['velocity'].shape)
processed_data['power'] = superdarn_data['power'] + xp.random.normal(0, 0.5, superdarn_data['power'].shape)

# Define validation metrics
validation_metrics = {
    'correlation': 0.95,
    'rmse_quality': 0.88,
    'bias_quality': 0.92,
    'completeness': 0.87,
    'temporal_consistency': 0.83
}

print("✅ Creating validation dashboard...")

# Create validation viewer
validator = ValidationViewer(superdarn_data, processed_data, validation_metrics)

# Also create validation dashboard
validation_fig = create_validation_dashboard(
    superdarn_data,
    processed_data, 
    validation_metrics,
    title="Processing Validation - Quality Assessment"
)

plt.show()

print("🔍 Validation analysis complete!")
print("   • Statistical comparison between original and processed data")
print("   • Correlation analysis and error metrics")
print("   • Quality score assessment")
print("   • Validation summary and recommendations")

## 9. Summary and Next Steps

This notebook has demonstrated the comprehensive visualization system for SuperDARN GPU processing:

In [ ]:
print("🎉 SuperDARN Visualization Tutorial Complete!")
print("=" * 50)
print()
print("📊 What we've covered:")
print("   ✅ Basic scientific visualization (range-time plots, fan plots)")
print("   ✅ Interactive data exploration with sliders and controls")
print("   ✅ Processing pipeline visualization (step-by-step)")
print("   ✅ Real-time performance monitoring")
print("   ✅ Comprehensive processing dashboards")
print("   ✅ Algorithm comparison and parameter tuning")
print("   ✅ Data quality validation and assessment")
print()
print("🚀 Next steps for scientists:")
print("   • Apply these tools to your real SuperDARN data")
print("   • Use interactive exploration to understand data patterns")
print("   • Monitor GPU performance to optimize processing")
print("   • Validate processing results with quality metrics")
print("   • Compare different algorithms and parameters")
print()
print("💡 Key benefits:")
print("   • Real-time insight into processing pipeline")
print("   • Interactive exploration for better understanding")
print("   • Performance optimization for faster processing")
print("   • Quality assurance for reliable results")
print("   • Publication-ready scientific visualizations")
print()
print("📚 For more examples, see:")
print("   • examples/demo_processing_visualization.py")
print("   • docs/visualization_guide.md")
print("   • benchmarks/performance_analysis.py")